In [ ]:
import re
import ollama
from rich.pretty import pprint
from dotenv import load_dotenv

from mlsgpt.dbv2 import store, models, schema
load_dotenv("../.env-deploy", override=True)

In [ ]:
reader = store.DataReader()
props = [models.Property.model_validate(prop) for prop in reader.get_properties()]

In [ ]:
system_prompt = open("summary_system_prompt.md").read()
prompt_template = "Description JSON:\n```json\n{}\n```"

In [ ]:
def parse_text(text: str) -> str:
    extracted = ""
    match = re.search(r'text\n(.*?)', text, re.DOTALL) 
    if match: 
        extracted = match.group(1) 
        return extracted

In [ ]:
def summarize(prop:models.Property) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_template.format(prop.model_dump_json())}
    ]
    response = ollama.chat(messages=messages, model="mistral")
    return response

In [ ]:
prop = models.Property.model_validate(props[0])
response = summarize(prop)

In [ ]:
# print(response["message"]["content"])

In [ ]:
# import concurrent.futures

# # Define a function to process a batch of embeddings
# def process_batch(batch):
#     with concurrent.futures.ThreadPoolExecutor(100) as executor:
#         futures = []
#         for data in batch:
#             futures.append(executor.submit(summarize, data))

#         results = []
#         for future in concurrent.futures.as_completed(futures):
#             results.append(future.result())
#     return results

# # Split the 'to_embed' list into batches of 1000
# batch_size = 100
# size = len(props)
# batches = [props[i:i+batch_size] for i in range(0, size, batch_size)]

# _schema = T.StructType(
#     [
#         T.StructField("ListingID", T.StringType(), False),
#         T.StructField("Summary", T.TextType(), False),
#         T.StructField("Embedding", T.StringType(), False)
#     ]
# )

# print(f"Processing {len(props)} records in {len(batches)} batches of {batch_size} records each")
# total_processed = 0
# for i, batch in enumerate(batches):
#     if i +  1 > 275:
#         rows = process_batch(batch)
#         df = spark.createDataFrame(rows, _schema)
#         df.write.csv(f"{data_home}/summaries/batch{str(i+1).zfill(6)}.csv", mode="overwrite", header=True)
#         total_processed += len(batch)
#         print(f"Processed batch {i+1} of {len(batches)} for ({total_processed} of {size}) records")